In [15]:
from urllib import request
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import math
import pandas as pd

terms=[]
url_list=['https://en.wikipedia.org/wiki/Web_mining', 'https://en.wikipedia.org/wiki/Data_mining', 'https://en.wikipedia.org/wiki/Machine_learning', 'https://en.wikipedia.org/wiki/Mining', 'https://en.wikipedia.org/wiki/Artificial_intelligence']
for url in url_list:
    html=request.urlopen(url).read().decode('utf8')
    soup=BeautifulSoup(html, 'html.parser')
    l=[]
    terms1=[]
    for titles in soup.find_all(['h1', 'h2','h3','h4','h5','h6', 'p']):
        l=word_tokenize(titles.text)
        terms1=terms1+l
    terms.append(terms1)
    
stop_words=stopwords.words('english')
stop_words_add=[ ',','.', '-', '(', ')', '[', ']', ':', ';', '\'','&','_']
for words in stop_words_add:
    stop_words.extend(words)

terms_without_stop=[]
for i in terms:
    l1=[]
    for w in i:
        if w not in stop_words:
            w=w.lower()
            l1.append(w)
    terms_without_stop.append(l1)

lemm=WordNetLemmatizer()
final_terms=[]
for i in terms_without_stop:
    l=[]
    for w in i:
        l.append(lemm.lemmatize(w))
    final_terms.append(l)

fopen=open('1.doc','w', encoding='utf-8')
for element in final_terms[0]:
    fopen.write(element)
    fopen.write('\n')
fopen.close()
fopen=open('2.doc','w', encoding='utf-8')
for element in final_terms[1]:
    fopen.write(element)
    fopen.write('\n')
fopen.close()
fopen=open('3.doc','w', encoding='utf-8')
for element in final_terms[2]:
    fopen.write(element)
    fopen.write('\n')
fopen.close()
fopen=open('4.doc','w', encoding='utf-8')
for element in final_terms[3]:
    fopen.write(element)
    fopen.write('\n')
fopen.close()
fopen=open('5.doc','w', encoding='utf-8')
for element in final_terms[4]:
    fopen.write(element)
    fopen.write('\n')
fopen.close()

common_list=[]
for i in final_terms:
    for w in i:
        if w not in common_list:
            common_list.append(w)

bf=[[],[],[],[],[]]
elements_count=[]
for i in final_terms:
    elements_count1={}
    for element in i:
       if element in elements_count1:
          elements_count1[element] += 1
       else:
          elements_count1[element] = 1
    elements_count.append(elements_count1)
        
for w in common_list:
    for i in range(0,len(elements_count)):
        if w in elements_count[i]:
            bf[i].append(elements_count[i][w])
        else:
            bf[i].append(0)

print("Bag of Words:")
df = pd. DataFrame([bf[0],bf[1],bf[2],bf[3],bf[4]], columns = common_list)
print(df)

common=[]
for i in final_terms:
    for w in i:
        if w not in common:
            common.append(w)

bf=[[],[],[],[],[]]
elements_count=[]
for i in final_terms:
    elements_count1={}
    for element in i:
       if element in elements_count1:
          elements_count1[element] += 1
       else:
          elements_count1[element] = 1
    elements_count.append(elements_count1)
        
for w in common:
    for i in range(0,len(elements_count)):
        if w in elements_count[i]:
            bf[i].append(elements_count[i][w])
        else:
            bf[i].append(0)

df = pd. DataFrame([bf[0],bf[1],bf[2],bf[3],bf[4]], columns = common)
print(df)



df=df.rename(index={0:"Doc1",1:"Doc2",2:"Doc3",3:"Doc4",4:"Doc5"})
print(df)



common.append("total words")
s=[0,0,0,0,0]

for i in range(0,len(bf)):
    s[i]+=sum(bf[i])
        
for i in range(0,len(bf)):
    for j in range(0,len(bf[i])):
        bf[i][j]=bf[i][j]/s[i]
        
for i in range(0,len(bf)):
    bf[i].append(s[i])
print("TF of Documents")
tf = pd.DataFrame([bf[0],bf[1],bf[2],bf[3],bf[4]], columns = common)
tf=tf.rename(index={0:"Doc1",1:"Doc2",2:"Doc3",3:"Doc4",4:"Doc5"})
print(tf)


idf=[]
for i in range(0,((len(common))-1)):
    c=0
    for j in range(0,len(bf)):
        if bf[j][i]!=0:
            c+=1
    idf.append(math.log((1+5)/c))
print('IDF: ')
print(idf)



for i in bf:
    for j in range(0,(len(i))-1):
        i[j]*=idf[j]
print("TF-IDF: ")
tfidf = pd.DataFrame([bf[0],bf[1],bf[2],bf[3],bf[4]], columns = common)
tfidf=tfidf.rename(index={0:"Doc1",1:"Doc2",2:"Doc3",3:"Doc4",4:"Doc5"})
print(tfidf)



s=input("Enter the query to be searched in the given documents: ")
s1=word_tokenize(s)
final_q=[]
for w in s1:
    if w not in stop_words:
        w=w.lower()
        final_q.append(lemm.lemmatize(w))
print("Processed Query: ")
print(final_q)
size=len(common)-1
tf_q=[]
for w in common:
    if w in final_q:
        tf_q.append(1/size)
    else:
        tf_q.append(0)

tfidf_q=[]
for i in range(0,len(common)-1):
    if common[i] in final_q:
        tfidf_q.append(tf_q[i]*idf[i])
    else:
        tfidf_q.append(0)
print('TF-IDF of Query: ')
print(tfidf_q)



n=0
for i in tfidf_q:
    n+=i**2
n=math.sqrt(n)
for i in tfidf_q:
    if i!=0:
        i=i/n
common.pop()
n_tfidf_q = pd.DataFrame([tfidf_q], columns = common)
n_tfidf_q=n_tfidf_q.rename(index={0:"Query"})
print('Normalised TF-IDF of Query: ')
print(n_tfidf_q)



common.append("Total Words")
nd=[0,0,0,0,0]
for i in range(0,5):
    for j in bf[i]:
        nd[i]+=j**2
for i in range(0,5):
    nd[i]=math.sqrt(nd[i])
for i in range(0,5):
    for j in bf[i]:
        j=j/nd[i]
n_tfidf = pd.DataFrame([bf[0],bf[1],bf[2],bf[3],bf[4]], columns = common)
n_tfidf=n_tfidf.rename(index={0:"Doc1",1:"Doc2",2:"Doc3",3:"Doc4",4:"Doc5"})
print("Normalised TF-IDF of Document Corpus")
print(n_tfidf)



cs={}
for i in range(0,5):
    s=0
    for j in range(0,len(common)-1):
        s+=(bf[i][j]*tfidf_q[j])
    cs[s]=i

for i in cs:
    print('Cosine Similarity of Doc {} with respect to the query: {}'.format(cs[i]+1, i))



ed={}
for i in range(0,5):
    s=0
    for j in range(0,len(common)-1):
        s+=((bf[i][j]-tfidf_q[j])**2)
    s=math.sqrt(s)
    ed[s]=i

for i in ed:
    print('Euclidean Distance of Doc {} with respect to the query: {}'.format(ed[i]+1, i))



print('Order of Documents from Euclidean Distance: ')
for i in sorted(ed.keys()):
    print('Doc {}'.format(ed[i]+1))



for i in range(0,4):
    for j in range(i+1,5):
        ed=0
        cs=0
        for k in range(0,len(common)-1):
            ed+=((bf[i][k]-bf[j][k])**2)
            cs+=bf[j][k]*bf[i][k]
        ed=math.sqrt(ed)
        print("Document Similarity between Doc {} and Doc {}: \nEuclidean Distance: {} and Cosine Similarity: {}".format(i+1,j+1,ed,cs))







Bag of Words:
   web  mining  application  data  technique  discover  pattern  world  wide  \
0   60      54            7    35          8         4        9      3     3   
1    1      76            8   138          2         0       22      1     0   
2    1      12            9    84         10         2        7      0     1   
3    0     185            4     0         10         0        0     25     0   
4    3       4           11    22          7         0        3      9     1   

    a  ...  android  dream  sheep  k.  dick  subjectivity  altered  272  \
0   3  ...        0      0      0   0     0             0        0    0   
1   9  ...        0      0      0   0     0             0        0    0   
2  15  ...        0      0      0   0     0             0        0    0   
3  11  ...        0      0      0   0     0             0        0    0   
4  28  ...        1      1      1   1     2             1        1    1   

   explanatory  fictional  
0            0          0 

In [13]:
stop_words=stopwords.words('english')
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '